## Modelo de datos del foro de debate

Este modelo de datos representa un foro de debate. Con este modelo los clientes pueden interaccionar con la comunidad de desarrolladores, plantear preguntas y contestar a las publicaciones de otros clientes. Cada servicio de AWS tiene un foro específico. Cualquier persona puede iniciar un nuevo hilo de debate publicando un mensaje en un foro, y cada hilo recibe un cierto número de respuestas.

Los patrones de acceso que este modelo de datos facilita son:

* Recuperación de un registro de foro utilizando el nombre del foro, facilitada por una tabla llamada Forum.

* Recuperación de un hilo específico o todos los hilos de un foro, facilitada por una tabla llamada Thread.

* Busque respuestas utilizando la dirección de correo electrónico del usuario de publicación, facilitado por el índice secundario global de la tabla Respuesta llamado PostedBy-Message-Index.

<br>

TABLA FORUM
<br>

![image.png](images/forum_data01.png)

<br>
TABLA THREAD
<br>

![image.png](images/forum_data02.png)

<br>
TABLA REPLY
<br>

![image.png](images/forum_data03.png)

<br>
GSI POSTEDBY-MESSAGE-INDEX
<br>

![image.png](images/forum_data04.png)

In [1]:
import pandas as pd
from spdynamodb import DynamoTable
import json
from decimal import Decimal
from time import sleep
import boto3

In [2]:
dt = DynamoTable()
try:
    dt.select_table('ForumData')
    print(dt)
except:
    dt.create_table(
        table_name='ForumData',
        partition_key='ForumName',
        partition_key_type='S',
        sort_key="Subject",
        sort_key_type="S"
)

In [3]:
dt.load_json("forum.json")

In [5]:
dt.create_global_secondary_index(
    att_name="PostedBy",
    att_type="S",
    sort_index="Subject",
    sort_type="S",
    i_name="PostedBy-Subject-Index",
    proj_type=["Message", "ReplyDateTime"],
    i_rcu=1,
    i_wcu=1
)

In [6]:
status = dt.check_status_gsi()
while status == 'CREATING':
    status = dt.check_status_gsi()
    sleep(30)
print("Global secondary index created.")

Global secondary index created.


In [6]:
dt.create_global_secondary_index(
    att_name="ForumName",
    att_type="S",
    sort_index="Title",
    sort_type="S",
    i_name="Thread-Index",
    i_rcu=10,
    i_wcu=10
)

In [ ]:
status = dt.check_status_gsi()
while status == 'CREATING':
    status = dt.check_status_gsi()
    sleep(30)
print("Global secondary index created.")

In [8]:
dt.create_global_secondary_index(
    att_name="Subject",
    att_type="S",
    sort_index="ReplyDateTime",
    sort_type="S",
    i_name="Reply-Date-Index"
)

In [ ]:
status = dt.check_status_gsi()
while status == 'CREATING':
    status = dt.check_status_gsi()
    sleep(30)
print("Global secondary index created.")

In [8]:
dt.check_status_gsi()

'CREATING'

In [4]:
dt.query_partiql("SELECT * FROM <table> WHERE Subject = ?", parameters=["FO-CAT"])

,Threads,Category,Messages,Views,ForumName,Subject
0,5,Amazon Web Services,5,667,Amazon Simple Notification Service,FO-CAT
1,22,Amazon Web Services,7,325,Amazon Simple Queue Service,FO-CAT
2,12,Amazon Web Services,33,405,Amazon MQ,FO-CAT
3,28,Amazon Web Services,12,412,AWS Data Pipeline,FO-CAT
4,55,Amazon Web Services,66,955,Amazon Athena,FO-CAT
5,30,Amazon Web Services,11,558,AWS Step Functions,FO-CAT
6,5,Amazon Web Services,13,225,Amazon EMR,FO-CAT
7,2,Amazon Web Services,4,405,Amazon DynamoDB,FO-CAT
